# Demo


## 0: Set-Up

Please run server before notebook:'

```
source .hamer/bin/activate
/home/gomer/hamer/run_server.py
```

In [ ]:
%load_ext autoreload
%autoreload 2


import sys
sys.path.append('/home/gomer/oline')
print(sys.path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['/home/gomer/miniconda3/envs/openhands/lib/python311.zip', '/home/gomer/miniconda3/envs/openhands/lib/python3.11', '/home/gomer/miniconda3/envs/openhands/lib/python3.11/lib-dynload', '', '/home/gomer/miniconda3/envs/openhands/lib/python3.11/site-packages', '/home/gomer/oline', '/home/gomer/oline', '/home/gomer/oline']


## 1: Data Preprocessing

### 1.1 Convert Videos to Hamer

In [2]:
from PoseTools.data.parsers_and_processors.converters.vid_to_hamer import main


input_folder = "/mnt/fishbowl/gomer/oline/sb_uva/videos"
output_folder = "/home/gomer/signbank_videos" #/mnt/fishbowl/gomer/oline/hamer_pkl"

#main(input_folder, output_folder + 'hamer_files')

### 1.2: Convert Videos to Pose Format

In [ ]:
from PoseTools.data.parsers_and_processors.converters.pose_to_pkl import main

#!videos_to_poses --format mediapipe --directory "/mnt/fishbowl/gomer/oline/sb_uva/videos"

#main(input_folder, output_folder + 'pose_files', delete_pose=False)


  0%|                                                  | 0/1502 [00:00<?, ?it/s]Loading video ...
Estimating pose ...
I0000 00:00:1731062973.873918  315265 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1731062973.925531  315306 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 550.90.12), renderer: NVIDIA TITAN RTX/PCIe/SSE2

0it [00:00, ?it/s]INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1731062974.153045  315282 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1731062974.222348  315300 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1731062974.226468  315286 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00

In [ ]:
import shutil
from pathlib import Path

def move_pose_files(source_dir, dest_dir):
    """
    Move all .pose files from the source directory to the destination directory.
    
    Parameters:
    source_dir (str or Path): Path to the source directory containing .pose files.
    dest_dir (str or Path): Path to the destination directory to move .pose files to.
    """
    source_dir = Path(source_dir)
    dest_dir = Path(dest_dir)
    dest_dir.mkdir(parents=True, exist_ok=True)  # Ensure the destination directory exists

    # Loop through all .pose files in the source directory
    for pose_file in source_dir.glob("*.pose"):
        # Move each .pose file to the destination directory
        shutil.move(pose_file, dest_dir / pose_file.name)
        print(f"Moved: {pose_file.name} to {dest_dir}")

move_pose_files(input_folder, output_folder)

## 1.3 Load Metadata

In [29]:
from PoseTools.data.parsers_and_processors.parsers import MetadataParser

metadata_path = '/home/gomer/oline/PoseTools/data/metadata/glosses_meta.json'
metadata = MetadataParser(metadata_path).read_metadata()
metadata

,Gloss ID,Annotation ID Gloss: Dutch,Annotation ID Gloss: English,Handedness,Strong Hand,Weak Hand,Affiliation
0,3808,#A,#A,1,A,-1,[Radboud]
1,3809,#B,#B,1,B,-1,[Radboud]
2,3810,#C,#C,1,C,-1,[Radboud]
3,3811,#D,#D,1,1,S,[Radboud]
4,3812,#E,#E,1,E,-1,[Radboud]
...,...,...,...,...,...,...,...
6887,49306,BACKPACKEN,BACKPACKING,2s,C,C,[UvA]
6888,49307,VAKANTIEDISCOUNTER,VAKANTIEDISCOUNTER,-1,-1,-1,[UvA]
6889,49308,VERWARD-C,CONFUSED-C,2s,1,1,[UvA]
6890,49309,STEM-AF-C,WITHOUT-A-VOICE-C,-1,-1,-1,[UvA]


## 2.0 Module 1 - Evaluate Handedness

This works if we only consider uncropped lexical video where the starting position is static 
Or it appears to work if there is only one hand in the video 
It might also work if the one hand starts a lot higher than the other



In [51]:
from PoseTools.src.modules.handedness.utils.eval_1_2_hands import main_handedness, evaluate_handedness

handedness_dict = main_handedness(output_folder, gloss_df=metadata, percent_threshold=30, test=True)
handedness_dict

Processing Pose Files: 100%|██████████| 6196/6196 [15:29<00:00,  6.67it/s]  


{'1900-B': ['R', 95.06],
 'PEUTER': ['R', 95.35],
 'BAL-B': ['B', 19.0],
 'VLAANDEREN': ['R', 81.02],
 'STROMING': ['B', 20.42],
 'BLIND-E': ['R', 98.98],
 'NIETMACHINE': ['B', 7.45],
 '67-A': ['R', 100.0],
 'TOCH-B': ['R', 88.5],
 'BALIE-B': ['B', 1.95],
 'LOPEN-B': ['B', 3.22],
 'KANKER-A': ['B', 4.95],
 'SLAAN-B': ['R', 97.94],
 'BIJVOORBEELD-A': ['R', 99.78],
 'UITKNIJPEN-TUBE': ['B', 2.53],
 'STRING-A': ['R', 97.75],
 'NAAMGEBAAR-D': ['B', 4.53],
 'INKTVIS-B': ['B', 17.39],
 'VASTHOUDEN-E': ['B', 17.99],
 'GRAS-C': ['B', 1.48],
 'CONSULTATIEBUREAU': ['R', 53.17],
 'PANIEK-A': ['B', 4.98],
 'FYSIEK': ['B', 3.21],
 'LIANNE-WESTENBERG': ['R', 94.1],
 'GENERATIE-A': ['B', 3.85],
 'CONTEXT-B': ['B', 0.77],
 '1000-C': ['R', 100.0],
 'SURINAME-B': ['R', 98.44],
 'STROOPWAFEL-C': ['R', 40.92],
 'HOND-B': ['R', 95.91],
 'VERZOEKEN': ['R', 93.13],
 'POES-C': ['L', 98.62],
 'STATION-A': ['R', 97.95],
 'HILVERSUM': ['B', 12.6],
 'KANO-B': ['R', 42.63],
 'PRAXIS': ['R', 94.21],
 'ITALIE-B': ['

In [ ]:
import json

def save_dict_to_txt(dictionary, filename):
    """
    Save a dictionary to a txt file in JSON format.

    Parameters:
    - dictionary (dict): The dictionary to save.
    - filename (str): Path to the output .txt file.
    """
    with open(filename, 'w') as file:
        json.dump(dictionary, file, indent=4)  # Pretty-print with indentation

# Example usage
output_filename = '/home/gomer/oline/PoseTools/src/modules/demo/results/handedness_results2.txt'
save_dict_to_txt(handedness_dict, output_filename)


In [47]:
# Example usage
# Assuming `handedness_dict` is already created and `metadata` is the DataFrame with ground truth data
accuracy, mismatches = evaluate_handedness(handedness_dict, metadata)

# Print mismatches for further analysis
#if mismatches:
#    print("Mismatches:")
#    for sign, details in mismatches.items():
#        print(f"{sign}: Predicted = {details['predicted']}, Expected = {details['expected']}")


NameError: name 'percent_difference' is not defined

## 2.1 Module 2 - Determine Handshape 

In [61]:
from PoseTools.src.modules.handshapes.engines.ed_algorithm.inference_dir import main_handshape
input_folder = '/home/gomer/oline/PoseTools/src/modules/demo/demo_files/hamer_files'
output_folder = '/home/gomer/oline/PoseTools/src/modules/demo/demo_files/graphics/handshapes'
ground_truth_labels = {'1': '5', 'A': 'B'}
main_handshape(input_folder, output_folder, ground_truth_labels)

Reference poses loaded: ['Q', 'C_spread', 'M', 'P', 'A', 'I', 'Beak_open_spread', 'Y', '1', 'N', '5r', 'C2_spread', 'Baby_C', '5m_closed', 'ILY', 'C', 'Beak', 'O', '5m', 'K', 'V_curved', 'Beak_spread', 'B', 'D', 'Baby_beak_open', 'Money', 'Baby_beak', '5', 'L2', 'L', 'R', '3', 'T', '5r_closed', '4', 'Beak_open', 'V', 'Baby_O', '1_curved', 'Beak2_open_spread', 'Horns', 'W', 'S', 'E', 'Flower']
Number of reference poses: 45

Confusion Matrix:


ValueError: 'labels' should contains at least one label.